In [2]:
import pandas as pd
import numpy as np
from openai import OpenAI
import joblib
from tqdm.notebook import tqdm
from glob import glob
import random
import json
from os import path, listdir

In [3]:
path_dir = '/home/crazyjeannot/Documents/doctorat/AVENTURES/data/booknlp/BOOKNLP_ENTITIES/'
path_test = '/home/crazyjeannot/Documents/doctorat/AVENTURES/data/booknlp/BOOKNLP_TEST/BOOKNLP_ENTITIES_TEST/'

path_entities = '/home/crazyjeannot/Documents/doctorat/AVENTURES/data/booknlp/BOOKNLP_ENTITIES/*.entities'
path_test_entities = '/home/crazyjeannot/Documents/doctorat/AVENTURES/data/booknlp/BOOKNLP_TEST/BOOKNLP_ENTITIES_TEST/*.entities'
path_chunk_entities = '/home/crazyjeannot/Documents/doctorat/AVENTURES/data/booknlp/BOOKNLP_TEST/1830_Sue-Eugene_Kernok-le-Pirate.txt_chunk_0.entities'

path_book = '/home/crazyjeannot/Documents/doctorat/AVENTURES/data/booknlp/BOOKNLP_BOOK/*.book'
path_test_book = '/home/crazyjeannot/Documents/doctorat/AVENTURES/data/booknlp/BOOKNLP_TEST/BOOKNLP_BOOK_TEST/*.book'
path_chunk_book = '/home/crazyjeannot/Documents/doctorat/AVENTURES/data/booknlp/BOOKNLP_TEST/1830_Sue-Eugene_Kernok-le-Pirate.txt_chunk_0.book'

In [4]:
def gpt_embeddings(chunk):
    client = OpenAI(
        api_key="[OPEN-AI-KEY]",
    )

    text = chunk.replace("\n", " ")
    embedding = client.embeddings.create(input = [text], model="text-embedding-ada-002").data[0].embedding
    
    return embedding

In [5]:
def get_characterization(booknlp_data):
    list_main = []
    for i in range(len(booknlp_data["characters"])):
        list_main.extend([item['w'] for item in booknlp_data["characters"][i]['agent']])
        list_main.extend([item['w'] for item in booknlp_data["characters"][i]['patient']])
        list_main.extend([item['w'] for item in booknlp_data["characters"][i]['mod']])
    return list_main

In [6]:
def get_embeddings(path_entities, path_book):
    categories_to_filter = ['GPE', 'LOC', 'VEH', 'TIME', 'FAC']
    columns_df = ['doc']+[i for i in range(0,1536)]
    df_main = pd.DataFrame(columns=columns_df)

    entities_files = sorted(glob(path_entities))
    book_files = sorted(glob(path_book))
    
    for doc_entities, doc_book in tqdm(zip(entities_files, book_files), total=len(listdir(path_dir))):
        file_entities = path.splitext(path.basename(doc_entities))[0]
        file_book = path.splitext(path.basename(doc_book))[0]
        
        if  file_entities == file_book:
        
            # GET ENTITIES / CHRONOTOPE
            df_entities = pd.read_csv(doc_entities, delimiter="\t")            
            df_chronotope = df_entities[df_entities['cat'].isin(categories_to_filter)]
            chronotope = list(df_chronotope.text)

            # GET CHARACTERIZATION
            with open(doc_book, "r") as booknlp_book:
                book_data = json.load(booknlp_book)
                characterization = get_characterization(book_data)

        else: 
            print(doc_entities)
            print(doc_book)
            print("NOT THE SAME FILE")
            break
            
        #print(f"len characterization = {len(characterization)}, len chronotope = {len(chronotope)}")

        characterization.extend(chronotope)
        random.shuffle(characterization)

        booknlp_str = ' '.join(characterization)
        embedding = gpt_embeddings(booknlp_str)
            
        new_row_data = [file_entities] + embedding    
        new_row_df = pd.DataFrame([new_row_data], columns=columns_df)
            
        df_main = pd.concat([df_main, new_row_df], ignore_index=True)            
            
        df_temp = df_main.set_index('doc')       
        df_temp.to_csv('GPT_EMBEDDINGS_BOOKNLP_OUTPUT_TEMP.csv')
    
    df_main.set_index('doc', inplace=True)
    df_main.to_csv('GPT_EMBEDDINGS_BOOKNLP_OUTPUT_MAIN.csv')
    
    return df_main

In [ ]:
df_res = get_embeddings(path_entities, path_book)

In [ ]:
df_res

In [7]:
df = pd.read_csv('GPT_EMBEDDINGS_BOOKNLP_OUTPUT_TEMP.csv')